<center><img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fwww.icicibank.com%2Fmanaged-assets%2Fimages%2Fblog%2Fbig%2Fadvantages-applying-personal.jpg&f=1&nofb=1"></center>

In this kernel i have dedscribed how one can do data cleaning of the data set and perform some univariate and bivariate analysis on this dataset.I will be updating the content of the kernel in future to make it more insightful

<font color="red" size=3>Please upvote this kernel if you like it. It motivates me to improve and produce more quality content :)

# Contents

* [<font size=4>Feature analysis</font>](#1)

    * [Data cleaning](#1.1)
    * [Univariate Analysis](#1.2)
    * [Bivariate Analysis](#1.3)
    
    
* [<font size=4>Modeling</font>](#2)

    * [TSNE-PCA visulization](#2.1)
    * [Model Training with no param tuning](#2.2)
    * [Param tuning](#2.3)
    * [Learning Curve and Feature Importance](#2.4)
    * [Stacked Classifier Glimpse](#2.5)

In [ ]:
import numpy as np
print('numpy version\t:',np.__version__)
import pandas as pd
print('pandas version\t:',pd.__version__)
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import stats

# Regular expressions
import re

# seaborn : advanced visualization
import seaborn as sns
print('seaborn version\t:',sns.__version__)

pd.options.mode.chained_assignment = None #set it to None to remove SettingWithCopyWarning
pd.options.display.float_format = '{:.4f}'.format #set it to convert scientific noations such as 4.225108e+11 to 422510842796.00
pd.set_option('display.max_columns', 100) # to display all the columns

np.set_printoptions(suppress=True,formatter={'float_kind':'{:f}'.format})

import os

import warnings
warnings.filterwarnings('ignore') # if there are any warning due to version mismatch, it will be ignored

In [ ]:
loan=pd.read_excel('/kaggle/input/task1/loan.xlsx')
loan.head()

In [ ]:
data_dict = pd.read_excel('/kaggle/input/task1/Data_Dictionary.xlsx')
data = data_dict.dropna()

In [ ]:
from IPython.display import display, HTML

display(HTML(data.to_html()))

#  Feature analysis<a id="1"></a>
## Data cleaning <a id="1.1"></a>

### Though the first step is to describe what our data signifies but first we will remove the columns that are around 30% null

SO WE WILL START WITH THE EDA FOR LOAN ANALYSIS

In [ ]:
print('What is the shape of the dataset:',loan.shape)

In [ ]:
import plotly.graph_objects as go
NA_col = loan.isnull().sum()
NA_col = NA_col[NA_col.values >(0.3*len(loan))]
d = pd.Series((NA_col.values/loan.shape[0])*100)

fig = go.Figure()
fig.add_trace(go.Bar(
    x=NA_col.index,
    y=NA_col.values,
    name='Null points',
    marker_color='indianred',
    text = d
))

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='group', xaxis_tickangle=-90)
fig.update_traces(texttemplate='%{text:.2s}', textposition='inside')
fig.show()

In [ ]:
def removeNulls(dataframe, axis =1, percent=0.7):
    '''
    * removeNull function will remove the rows and columns based on parameters provided.
    * dataframe : Name of the dataframe  
    * axis      : axis = 0 defines drop rows, axis =1(default) defines drop columns    
    * percent   : percent of data where column/rows values are null,default is 0.3(30%)
              
    '''
    df = dataframe.copy()
    ishape = df.shape
    if axis == 0:
        rownames = df.transpose().isnull().sum()
        rownames = list(rownames[rownames.values > percent*len(df)].index)
        df.drop(df.index[rownames],inplace=True) 
        print("\nNumber of Rows dropped\t: ",len(rownames))
    else:
        colnames = (df.isnull().sum()/len(df))
        colnames = list(colnames[colnames.values>=percent].index)
        df.drop(labels = colnames,axis =1,inplace=True)        
        print("Number of Columns dropped\t: ",len(colnames))
        
    print("\nOld dataset rows,columns",ishape,"\nNew dataset rows,columns",df.shape)

    return df

THIS IS DONE TO REMOVE COLUMNS THAT CONTAIN MORE THAN 30% DATA AS NULLs

In [ ]:
loan = removeNulls(loan, axis =1,percent = 0.3)

SO NOW WHAT WE CAN SEE THAT 56 COLUMNS ARE THE ONES WHICH ARE 100% NULL OR MORE THAN 92% SO THEY SHOULD BE REMOVED

SOME MORE COLUMNS ARE ELIMINATED BECAUSE OF NOT UNIQUE VALUES

In [ ]:
loan['issue_d_Year'] = loan['issue_d'].dt.year
loan['issue_d_Month'] = loan['issue_d'].dt.month
loan['issue_d_Day'] = loan['issue_d'].dt.day
loan['earliest_cr_line_Year'] = loan['earliest_cr_line'].dt.year
loan['earliest_cr_line_Month'] = loan['earliest_cr_line'].dt.month
loan['earliest_cr_line_Day'] = loan['earliest_cr_line'].dt.day
loan['last_credit_pull_d_Year'] = loan['last_credit_pull_d'].dt.year
loan['last_credit_pull_d_Month'] = loan['last_credit_pull_d'].dt.month
loan['last_credit_pull_d_Day'] = loan['last_credit_pull_d'].dt.day
loan['last_pymnt_d_Year'] = loan['last_pymnt_d'].dt.year
loan['last_pymnt_d_Month'] = loan['last_pymnt_d'].dt.month
loan['last_pymnt_d_Day'] = loan['last_pymnt_d'].dt.day
loan = loan.drop(columns=['issue_d','last_pymnt_d','last_credit_pull_d','earliest_cr_line'])

In [ ]:
unique = loan.nunique()
unique = unique[unique.values == 1]

In [ ]:
loan.drop(labels = list(unique.index), axis =1, inplace=True)
print("So now we are left with",loan.shape ,"rows & columns.")

In [ ]:
print(loan.emp_length.unique())
loan.emp_length.fillna('0',inplace=True)
loan.emp_length.replace(['n/a'],'Self-Employed',inplace=True)
print(loan.emp_length.unique())

In [ ]:
not_required_columns = ["id","member_id","url","zip_code"]
loan.drop(labels = not_required_columns, axis =1, inplace=True)
print("So now we are left with",loan.shape ,"rows & columns.")

In [ ]:
loan

In [ ]:
def plotting_3_chart(df, feature):
    ## Importing seaborn, matplotlab and scipy modules. 
    import seaborn as sns
    import matplotlib.pyplot as plt
    import matplotlib.gridspec as gridspec
    from scipy import stats
    import matplotlib.style as style
    style.use('fivethirtyeight')

    ## Creating a customized chart. and giving in figsize and everything. 
    fig = plt.figure(constrained_layout=True, figsize=(7,7))
    ## creating a grid of 3 cols and 3 rows. 
    grid = gridspec.GridSpec(ncols=3, nrows=3, figure=fig)
    #gs = fig3.add_gridspec(3, 3)

    ## Customizing the histogram grid. 
    ax1 = fig.add_subplot(grid[0, :2])
    ## Set the title. 
    ax1.set_title(feature + ' Histogram')
    ## plot the histogram. 
    sns.distplot(df.loc[:,feature], fit = stats.norm,norm_hist=True, ax = ax1)

    # customizing the QQ_plot. 
    ax2 = fig.add_subplot(grid[1, :2])
    ## Set the title. 
    ax2.set_title(feature + ' QQ_plot')
    ## Plotting the QQ_Plot. 
    stats.probplot(df.loc[:,feature], plot = ax2)

    ## Customizing the Box Plot. 
    ax3 = fig.add_subplot(grid[:, 2])
    ## Set title. 
    ax3.set_title(feature + ' Box Plot')
    ## Plotting the box plot. 
    sns.boxplot(df.loc[:,feature], orient='v', ax = ax3 )
    k2, p = stats.normaltest(df[feature].values)
    skewness = df[feature].skew()
    kurtosis = df[feature].kurtosis()
    print('\n kurtosis = %.3f,skew = %.3f ,p-value = %.3f' % (kurtosis,skewness,p))

    alpha = 0.05
    if p > alpha:
        print('\nThe transformed data is Gaussian (fails to reject the null hypothesis)')
    else:
        print('\nThe transformed data does not look Gaussian (reject the null hypothesis)')


In [ ]:
(loan.purpose.value_counts()*100)/len(loan)

In [ ]:
numeric_columns = ['loan_amnt','funded_amnt','funded_amnt_inv','installment','int_rate','annual_inc','dti']

loan[numeric_columns] = loan[numeric_columns].apply(pd.to_numeric)

In [ ]:
del_loan_purpose = (loan.purpose.value_counts()*100)/len(loan)
del_loan_purpose = del_loan_purpose[(del_loan_purpose < 0.75) | (del_loan_purpose.index == 'other')]
loan = loan.drop(columns=['emp_title'])
loan.drop(labels = loan[loan.purpose.isin(del_loan_purpose.index)].index, inplace=True)
print("So now we are left with",loan.shape ,"rows & columns.")

print(loan.purpose.unique())

In [ ]:
(loan.loan_status.value_counts()*100)/len(loan)

In [ ]:
del_loan_status = (loan.loan_status.value_counts()*100)/len(loan)
del_loan_status = del_loan_status[(del_loan_status < 1.5)]

loan.drop(labels = loan[loan.loan_status.isin(del_loan_status.index)].index, inplace=True)
print("So now we are left with",loan.shape ,"rows & columns.")

print(loan.loan_status.unique())

In [ ]:
loan['loan_income_ratio']= loan['loan_amnt']/loan['annual_inc']

In [ ]:
bins = [0, 5000, 10000, 15000, 20000, 25000,40000]
slot = ['0-5000', '5000-10000', '10000-15000', '15000-20000', '20000-25000','25000 and above']
loan['loan_amnt_range'] = pd.cut(loan['loan_amnt'], bins, labels=slot)

In [ ]:
bins = [0, 25000, 50000, 75000, 100000,6000000]
slot = ['0-25000', '25000-50000', '50000-75000', '75000-100000', '100000 and above']
loan['annual_inc_range'] = pd.cut(loan['annual_inc'], bins, labels=slot)

In [ ]:
bins = [0, .075, .10, .125, .15,.20,.25]
slot = ['0-7.5', '7.5-10', '10-12.5', '12.5-15', '15 -20','20-25']
loan['int_rate_range'] = pd.cut(loan['int_rate'], bins, labels=slot)

CREATING BINS FOR EASE OF PLOTTING

In [ ]:
quantitative = [f for f in loan.columns if loan.dtypes[f] != 'object']
qualitative = [f for f in loan.columns if loan.dtypes[f] == 'object']
qualitative.remove('loan_status')

## UNIVARIATE ANALYSIS <a id="1.2"></a>

STARTING WITH UNIVARIATE ANALYSIS


Continuous Variables

In case of continuous variables, we need to understand the CENTRAL TENDENCY and SPREAD of the variable.
These are measured using various statistical metrics visualization methods such as 
BOXPLOT,HISTOGRAM/DISTRIBUTION PLOT, VIOLIN PLOT etc.
<br>
<br>
Categorical Variables

<br>
For categorical variables, we’ll use frequency table to understand distribution of each category. 
It can be be measured using two metrics, COUNT and COUNT% against each category.
COUNTBARPLOT or BAR CHART can be used as visualization.


In [ ]:
def univariate(df,col,vartype,hue =None):
    
    '''
    Univariate function will plot the graphs based on the parameters.
    df      : dataframe name
    col     : Column name
    vartype : variable type : continuos or categorical
                Continuos(0)   : Distribution, Violin & Boxplot will be plotted.
                Categorical(1) : Countplot will be plotted.
    hue     : It's only applicable for categorical analysis.
    
    '''
    sns.set(style="darkgrid")
    
    if vartype == 0:
        plotting_3_chart(df,col)
    
    if vartype == 1:
        temp = pd.Series(data = hue)
        fig, ax = plt.subplots()
        width = len(df[col].unique()) + 6 + 4*len(temp.unique())
        fig.set_size_inches(width , 7)
        ax = sns.countplot(data = df, x= col, order=df[col].value_counts().index,hue = hue) 
        if len(temp.unique()) > 0:
            for p in ax.patches:
                ax.annotate('{:1.1f}%'.format((p.get_height()*100)/float(len(loan))), (p.get_x()+0.05, p.get_height()+20))  
        else:
            for p in ax.patches:
                ax.annotate(p.get_height(), (p.get_x()+0.32, p.get_height()+20)) 
        del temp
    else:
        exit
        
    plt.show()

In [ ]:
univariate(df=loan,col='loan_amnt',vartype=0)

Inference: Most of the loan amounts are distributed between 6000 to 15000

In [ ]:
univariate(df=loan,col='int_rate',vartype=0)

Inference: Most of the loans interest rates are distributed between 9.5% to 14.5%.

In [ ]:
univariate(df=loan,col='annual_inc',vartype=0)

Most of the applicants earns beteen 48000 to 85000 annually.

In [ ]:
univariate(df=loan,col='installment',vartype=0)

In [ ]:
univariate(df=loan,col='open_acc',vartype=0)

In [ ]:
univariate(df=loan,col='loan_amnt_range',vartype=1)

In [ ]:
univariate(df=loan,col='loan_status',vartype=1)

15% of the applicants Charged off

In [ ]:
univariate(df=loan,col='verification_status',vartype=1,hue='loan_status')

In [ ]:
univariate(df=loan,col='term',vartype=1,hue='loan_status')

In [ ]:
loan.home_ownership.unique()

In [ ]:
rem = ['OTHER', 'NONE', 'ANY']
loan.drop(loan[loan['home_ownership'].isin(rem)].index,inplace=True)
loan.home_ownership.unique()

In [ ]:
univariate(df=loan,col='home_ownership',vartype=1,hue='loan_status')

47% of applicants are living in rented home whereas 45.3% applicants were mortagaged their home 
and 7.5% in their own

## Bivariate Analysis  <a id="1.3"></a>

Bivariate/Multivariate Analysis

Bivariate/Multivariate Analysis finds out the relationship between two/two or more variables.
We can perform Bivariate/Multivariate analysis for any combination of categorical and continuous variables. 
The combination can be: Categorical & Categorical, Categorical & Continuous and Continuous & Continuous.


In [ ]:
plt.figure(figsize=(16,12))
sns.boxplot(data =loan, x='purpose', y='loan_amnt', hue ='loan_status')
plt.title('Purpose of Loan vs Loan Amount')
plt.show()

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Heatmap(
                    z=loan.corr(),x=loan.corr().columns
,y=loan.corr().columns))
fig.show()

It is clear from the Heatmap that how 'loan_amnt','funded_amnt' & 'funded_amnt_inv' are closely interrelated.
So we can take any one column out of them for our analysis

In [ ]:
loanstatus=loan.pivot_table(index=['loan_status','purpose','emp_length'],values='loan_amnt',aggfunc=('count')).reset_index()
loanstatus=loan.loc[loan['loan_status']=='Charged Off']

In [ ]:
ax = plt.figure(figsize=(30, 18))
ax = sns.boxplot(x='emp_length',y='loan_amnt',hue='purpose',data=loanstatus)
ax.set_title('Employment Length vs Loan Amount for different pupose of Loan',fontsize=22,weight="bold")
ax.set_xlabel('Employment Length',fontsize=16)
ax.set_ylabel('Loan Amount',color = 'b',fontsize=16)
plt.show()

In [ ]:
def crosstab(df,col):
    '''
    df : Dataframe
    col: Column Name
    '''
    crosstab = pd.crosstab(df[col], df['loan_status'],margins=True)
    crosstab['Probability_Charged Off'] = round((crosstab['Charged Off']/crosstab['All']),3)
    crosstab = crosstab[0:-1]
    return crosstab

Bivariate/Multivariate Analysis with Probability of Charge off
Categorical Variables vs Probability of Charged Off

The main motive of this use case to find what parameters are impacting the most on Loan Status 
that is if a applicant will successfully complete the loan term or will charge off.

So we will be using a new term now Probability of Charged Off that will be equal to :

ProbabilityofChargedOff=Number of Applicants who charged off / TotalNo.ofApplicants

We will calculate this probability w.r.t each column in bivariate analysis & will see how the 
Probability of Charged Off changes with these columns.

We will create a user defined function for this

In [ ]:
# Probability of charge off
def bivariate_prob(df,col,stacked= False):
    '''
    df      : Dataframe
    col     : Column Name
    stacked : True(default) for Stacked Bar
    '''
    # get dataframe from crosstab function
    plotCrosstab = crosstab(df,col)
    
    linePlot = plotCrosstab[['Probability_Charged Off']]      
    barPlot =  plotCrosstab.iloc[:,0:2]
    ax = linePlot.plot(figsize=(20,8), marker='o',color = 'b')
    ax2 = barPlot.plot(kind='bar',ax = ax,rot=1,secondary_y=True,stacked=stacked)
    ax.set_title(df[col].name.title()+' vs Probability Charge Off',fontsize=20,weight="bold")
    ax.set_xlabel(df[col].name.title(),fontsize=14)
    ax.set_ylabel('Probability of Charged off',color = 'r',fontsize=14)
    ax2.set_ylabel('Number of Applicants',color = 'g',fontsize=14)
    plt.show()

In [ ]:
filter_states = loan.addr_state.value_counts()
filter_states = filter_states[(filter_states < 10)]

loan_filter_states = loan.drop(labels = loan[loan.addr_state.isin(filter_states.index)].index)

In [ ]:
states = crosstab(loan_filter_states,'addr_state')
display(states.tail(20))

bivariate_prob(df =loan_filter_states,col ='addr_state')

There are multiple States/Provinces with high probability of charge,highest being 'NV' at 20%

In [ ]:
purpose = crosstab(loan,'purpose')
display(purpose)

bivariate_prob(df =loan,col ='purpose',stacked=False)

Applicants who has taken the Loan for 'small business' has the highest probabilty of charge off of 26%.
So bank should take extra caution like take some asset or guarentee 
while approving the loan for purpose of 'small business'

In [ ]:
grade = crosstab(loan,'grade')
display(grade)

bivariate_prob(df =loan,col ='grade',stacked=False)
bivariate_prob(df =loan,col ='sub_grade')

As we move from Grade A to G, probability that person will charged off is increasing.

In [ ]:
annual_inc_range = crosstab(loan,'annual_inc_range')
display(annual_inc_range)

bivariate_prob(df =loan,col ='annual_inc_range')

As the annual income is decreasing the probability 
that person will default is increasing with highest
of 18% at (0 to 25000) salary bracket.

In [ ]:
emp_length = crosstab(loan,'emp_length')
display(emp_length)

bivariate_prob(df =loan,col ='emp_length')

Applicants who are self employed & less than 1 year of experience are more probable of charged off.. 

In [ ]:
int_rate_range = crosstab(loan,'int_rate_range')
display(int_rate_range)

bivariate_prob(df =loan,col ='int_rate_range')

interest rate of 15% or above shows around 25% chance of charged off 
with 22.5% chances with interest rate of 10-12.5%

In [ ]:
corr_matrix = loan.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.6)]
loan.drop(to_drop, axis=1, inplace=True)

In [ ]:
loan['loan_amnt_range'] = loan['loan_amnt_range'].cat.codes
loan['annual_inc_range'] =loan['annual_inc_range'].cat.codes
loan['int_rate_range'] = loan['int_rate_range'].cat.codes
for i in qualitative:
    loan[i] = loan[i].astype('category').cat.codes
loan['loan_status'] = loan['loan_status'].astype('category').cat.codes

In [ ]:
loan

In [ ]:
quantitative = [f for f in loan.columns if loan.dtypes[f] != 'object' or loan.dtypes[f] != 'int8']
quantitative.remove('issue_d_Year')
quantitative.remove('issue_d_Month')
quantitative.remove('earliest_cr_line_Year')
quantitative.remove('earliest_cr_line_Month')
quantitative.remove('last_credit_pull_d_Year')
quantitative.remove('last_credit_pull_d_Month')
quantitative.remove('last_pymnt_d_Month')
quantitative.remove('loan_status')
a = set(quantitative)
b = set(qualitative)
a = a - a.intersection(b)
quantitative = a
quantitative = list(quantitative)

In [ ]:
from collections import Counter
def detect_outliers(df,n,features):
    """
    Takes a dataframe df of features and returns a list of the indices
    corresponding to the observations containing more than n outliers according
    to the Tukey method.
    """
    outlier_indices = []
    for col in features:
        # 1st quartile (25%)
        Q1 = np.percentile(df[col], 25)
        # 3rd quartile (75%)
        Q3 = np.percentile(df[col],75)
        # Interquartile range (IQR)
        IQR = Q3 - Q1
        
        # outlier step
        outlier_step = 1.5 * IQR
        
        # Determine a list of indices of outliers for feature col
        outlier_list_col = df[(df[col] < Q1 - outlier_step) | (df[col] > Q3 + outlier_step )].index
        
        # append the found outlier indices for col to the list of outlier indices 
        outlier_indices.extend(outlier_list_col)
        
    # select observations containing more than 2 outliers
    outlier_indices = Counter(outlier_indices)        
    multiple_outliers = list( k for k, v in outlier_indices.items() if v > n )
    
    return multiple_outliers   

#Outliers_to_drop = detect_outliers(df,2,df.columns.values)

### Pca-TSNE plots + Naive kmeans <a id="2.1"></a>

In [ ]:
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import *
model = TSNE(n_components=3, random_state=0, perplexity=50)
y = loan.loan_status.values
X = loan.drop(columns=['loan_status'])
X = X.fillna(0.)
std = StandardScaler()
X = std.fit_transform(X)
print('<-----------------------------TSNE + PCA on data--------------------------->')
pca = PCA(n_components=30)
pca.fit(X)
pc = pca.transform(X)
kmeans = KMeans(n_clusters=3)
kmeans.fit(pc)
y_res = kmeans.predict(pc) 
tsne1 = model.fit_transform(pc)
fr1 = pd.DataFrame({'tsne1': tsne1[:,0], 'tsne2': tsne1[:, 1], 'tsne3':tsne1[:,2],'cluster': y})
fr = pd.DataFrame({'tsne1': tsne1[:,0], 'tsne2': tsne1[:, 1], 'tsne3':tsne1[:,2],'cluster': y_res})
print('explained varience : with 30 components' , np.sum(pca.explained_variance_ratio_))

In [ ]:
import plotly.express as px
fig = px.scatter_3d(fr1, x='tsne1', y='tsne2', z='tsne3',
                    color='cluster',opacity=0.6)
fig.show()
fig = px.scatter_3d(fr, x='tsne1', y='tsne2', z='tsne3',
                    color='cluster',opacity=0.6)
fig.show()

#### upper one is plot of TSNE + PCA labelled with true Classes
#### lower one is same as upper one difference is of the points are labelled with kmeans prediction

In [ ]:
print('f1_Score of kmeans:', f1_score(y,y_res,average='weighted'))

### Training predefined params <a id="2.2"></a>

#### results of training on pca data

In [ ]:
from sklearn.linear_model import *
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve
from sklearn.pipeline import make_pipeline
from mlxtend.classifier import StackingCVClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

kfold = StratifiedKFold(n_splits=10)
# Modeling step Test differents algorithms 
random_state = 2
classifiers = []
classifiers.append(SVC(random_state=random_state))
classifiers.append(DecisionTreeClassifier(random_state=random_state))
classifiers.append(AdaBoostClassifier(DecisionTreeClassifier(random_state=random_state),random_state=random_state,learning_rate=0.01))
classifiers.append(RandomForestClassifier(random_state=random_state))
classifiers.append(ExtraTreesClassifier(random_state=random_state))
classifiers.append(GradientBoostingClassifier(random_state=random_state))
classifiers.append(MLPClassifier(random_state=random_state))
classifiers.append(KNeighborsClassifier())
classifiers.append(LogisticRegression(random_state = random_state))
classifiers.append(LinearDiscriminantAnalysis())
classifiers.append(XGBClassifier(random_state = random_state))
classifiers.append(LGBMClassifier(boosting_type = 'gbdt',objective= 'multiclass',num_class=3,random_state = random_state))

cv_results = []
for classifier in classifiers :
    print(classifier)
    cv_results.append(cross_val_score(classifier, pc, y = y, scoring = "f1_weighted", cv = kfold, n_jobs=6))

cv_means = []
cv_std = []
for cv_result in cv_results:
    cv_means.append(cv_result.mean())
    cv_std.append(cv_result.std())

cv_res = pd.DataFrame({"CrossValMeans":cv_means,"CrossValerrors": cv_std,"Algorithm":["SVC","DecisionTree","AdaBoost",
"RandomForest","ExtraTrees","GradientBoosting","MultipleLayerPerceptron","KNeighboors","LogisticRegression",
                                                                                      "LinearDiscriminantAnalysis","XGBClassifier","LGBMClassifier"]})

g = sns.barplot("CrossValMeans","Algorithm",data = cv_res, palette="Set3",orient = "h",**{'xerr':cv_std})
g.set_xlabel("Mean f1_weighted")
g = g.set_title("Cross validation scores")

In [ ]:
import plotly.graph_objects as go

months = cv_res.Algorithm.values

fig = go.Figure()
fig.add_trace(go.Bar(
    x=months,
    y=cv_res.CrossValerrors.values,
    name='CrossValerrors',
    marker_color='Blue'
))
fig.add_trace(go.Bar(
    x=months,
    y=cv_res.CrossValMeans.values,
    name='CrossValMeans',
    marker_color='lightsalmon'
))

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='group', xaxis_tickangle=-45)
fig.show()

#### result on regular data

In [ ]:
X = loan.drop(columns=['loan_status'])
X = X.fillna(0.)
X = RobustScaler().fit_transform(X)
cv_results = []
for classifier in classifiers :
    print(classifier)
    cv_results.append(cross_val_score(classifier, X, y = y, scoring = "f1_weighted", cv = kfold, n_jobs=6))

cv_means = []
cv_std = []
for cv_result in cv_results:
    cv_means.append(cv_result.mean())
    cv_std.append(cv_result.std())

cv_res = pd.DataFrame({"CrossValMeans":cv_means,"CrossValerrors": cv_std,"Algorithm":["SVC","DecisionTree","AdaBoost",
"RandomForest","ExtraTrees","GradientBoosting","MultipleLayerPerceptron","KNeighboors","LogisticRegression",
                                                                                      "LinearDiscriminantAnalysis","XGBClassifier","LGBMClassifier"]})

g = sns.barplot("CrossValMeans","Algorithm",data = cv_res, palette="Set3",orient = "h",**{'xerr':cv_std})
g.set_xlabel("Mean f1_weighted")
g = g.set_title("Cross validation scores")

months = cv_res.Algorithm.values

fig = go.Figure()
fig.add_trace(go.Bar(
    x=months,
    y=cv_res.CrossValerrors.values,
    name='CrossValerrors',
    marker_color='Blue'
))
fig.add_trace(go.Bar(
    x=months,
    y=cv_res.CrossValMeans.values,
    name='CrossValMeans',
    marker_color='lightsalmon'
))

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='group', xaxis_tickangle=-45)
fig.show()

### H-Param tuning <a id="2.3"></a>

In [ ]:
MLP = MLPClassifier((34,10,3),learning_rate_init=0.01,alpha=1e-5,validation_fraction=.1)

mlp_param_grid = {'solver': ['sgd','adam'], 
              'max_iter': [1100,800],
              'random_state':[2],
            'learning_rate':['constant', 'adaptive']
                 }

gsMlpC = GridSearchCV(MLP,param_grid = mlp_param_grid, cv=kfold, scoring="f1_weighted", n_jobs= -1, verbose = 1)

gsMlpC.fit(X,y)

MLP_best = gsMlpC.best_estimator_

# Best score
gsMlpC.best_score_

In [ ]:
# Adaboost
DTC = DecisionTreeClassifier()

adaDTC = AdaBoostClassifier(DTC, random_state=2)

ada_param_grid = {"base_estimator__criterion" : ["entropy"],
              "n_estimators" :[1,2],
              "learning_rate":  [0.001, 0.01, 0.1]}

gsadaDTC = GridSearchCV(adaDTC,param_grid = ada_param_grid, cv=kfold, scoring="f1_weighted", n_jobs= -1, verbose = 1)

gsadaDTC.fit(X,y)

ada_best = gsadaDTC.best_estimator_

In [ ]:
#ExtraTrees 
ExtC = ExtraTreesClassifier()


## Search grid for optimal parameters
ex_param_grid = {"max_depth": [None],
              "min_samples_split": [2, 10],
              "min_samples_leaf": [1,3],
              "bootstrap": [False],
              "n_estimators" :[100,300],
              "criterion": ["entropy"]}


gsExtC = GridSearchCV(ExtC,param_grid = ex_param_grid, cv=kfold, scoring="f1_weighted", n_jobs= -1, verbose = 1)

gsExtC.fit(X,y)

ExtC_best = gsExtC.best_estimator_

# Best score
gsExtC.best_score_

In [ ]:
# RFC Parameters tunning 
RFC = RandomForestClassifier()


## Search grid for optimal parameters
rf_param_grid = {"max_depth": [4,8],
              "min_samples_split": [2, 10],
              "min_samples_leaf": [3, 10],
              "n_estimators" :[150]}

gsRFC = GridSearchCV(RFC,param_grid = rf_param_grid, cv=kfold, scoring="f1_weighted", n_jobs= -1, verbose = 1)

gsRFC.fit(X,y)

RFC_best = gsRFC.best_estimator_

# Best score
gsRFC.best_score_

In [ ]:
# Gradient boosting tunning

GBC = GradientBoostingClassifier()
gb_param_grid = {'loss' : ["deviance"],
              'n_estimators' : [100,300],
              'learning_rate': [0.001, 0.01],
              'max_depth': [4, 8],
              }

gsGBC = GridSearchCV(GBC,param_grid = gb_param_grid, cv=kfold, scoring="f1_weighted", n_jobs= -1, verbose = 1)

gsGBC.fit(X,y)

GBC_best = gsGBC.best_estimator_

# Best score
gsGBC.best_score_

### Learning curve and Feature Importance  <a id="2.4"></a>

In [ ]:
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=-1, train_sizes=np.linspace(.1, 1.0, 5)):
    """Generate a simple plot of the test and training learning curve"""
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    print(estimator)
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt
X_train = X
Y_train = y
g = plot_learning_curve(gsRFC.best_estimator_,"RF mearning curves",X_train,Y_train,cv=kfold)
g = plot_learning_curve(gsExtC.best_estimator_,"ExtraTrees learning curves",X_train,Y_train,cv=kfold)
g = plot_learning_curve(gsadaDTC.best_estimator_,"AdaBoost learning curves",X_train,Y_train,cv=kfold)
g = plot_learning_curve(gsMlpC.best_estimator_,"Multilayer Perceptron",X_train,Y_train,cv=kfold)

In [ ]:
nrows = ncols = 2
X_train = X
fig, axes = plt.subplots(nrows = nrows, ncols = ncols, sharex="all", figsize=(15,15))
loans = loan1.drop(columns=['loan_status'])
X_train = pd.DataFrame(X)
X_train.columns = loans.columns
names_classifiers = [("AdaBoosting", ada_best),("ExtraTrees",ExtC_best),
                     ("RandomForest",RFC_best),("GradientBoosting",GBC_best)]

nclassifier = 0
for row in range(nrows):
    for col in range(ncols):
        name = names_classifiers[nclassifier][0]
        classifier = names_classifiers[nclassifier][1]
        indices = np.argsort(classifier.feature_importances_)[::-1]
        g = sns.barplot(y=X_train.columns[indices],x = classifier.feature_importances_[indices] , orient='h',ax=axes[row][col])
        g.set_xlabel("Relative importance",fontsize=12)
        g.set_ylabel("Features",fontsize=12)
        g.tick_params(labelsize=9)
        g.set_title(name + " feature importance")
        nclassifier += 1

### Glimpse of stacked classifier  <a id="2.5"></a>

In [ ]:
LGBC = LGBMClassifier(random_state = 2)
sclf = StackingCVClassifier(classifiers=classifiers[:-1],
                            meta_classifier=LGBC,
                            random_state=2)

scores = cross_val_score(sclf, X, y, cv=3, scoring='f1_weighted')
print("f1_weighted : %0.2f (+/- %0.2f) [%s]" 
          % (scores.mean(), scores.std(), 'stacked_classifier'))

<font color="red" size=3>Please upvote this kernel if you like it. It motivates me to improve and produce more quality content :)